In [1]:
!nvidia-smi

Tue Dec 12 14:35:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [3]:
!pip install --upgrade accelerate
!pip install -y transofrmers accelerate
!pip install transformers accelerate


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -y


In [4]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
import pandas as pd

from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
#dowload & unzip data

#!wget https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
#!unzip summarizer-data.zip

!unzip archive.zip

Archive:  archive.zip
replace BBCarticles_csv.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [7]:
import pandas as pd
data = pd.read_csv("BBCarticles_csv.csv",encoding = 'latin1')
data.head()

,Summary,Text
0,TimeWarner said fourth quarter sales rose 2% t...,Ad sales boost Time Warner profit\n\nQuarterly...
1,The dollar has hit its highest level against t...,Dollar gains on Greenspan speech\n\nThe dollar...
2,Yukos' owner Menatep Group says it will ask Ro...,Yukos unit buyer faces loan claim\n\nThe owner...
3,"Rod Eddington, BA's chief executive, said the ...",High fuel prices hit BA's profits\n\nBritish A...
4,Pernod has reduced the debt it took on to fund...,Pernod takeover talk lifts Domecq\n\nShares in...


In [8]:
data = data[:500]

In [9]:
data.shape

(500, 2)

In [10]:
from torch.utils.data import DataLoader, Dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_source_length=512, max_target_length=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data['Text'].iloc[idx]
        summary = self.data['Summary'].iloc[idx]

        # Tokenize text and summary using the Pegasus tokenizer
        encoding = self.tokenizer(
            text,
            truncation=True,
            max_length=128,
            return_tensors='pt',
            padding='max_length'
        )

        labels = self.tokenizer(
            summary,
            truncation=True,
            max_length=128,
            return_tensors='pt',
            padding='max_length'
        )

        return  {
        'input_ids': encoding['input_ids'].to(device),
        'attention_mask': encoding['attention_mask'].to(device),
        'labels': labels['input_ids'].to(device),
        'labels_attention_mask': labels['attention_mask'].to(device)
    }

In [11]:
from  sklearn.model_selection import train_test_split
x_train , x_test = train_test_split(data, test_size=0.1)

In [12]:
train_dataset = CustomDataset(x_train, tokenizer)
test_dataset = CustomDataset(x_test, tokenizer)

In [13]:
# DataLoader for train and test
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [14]:
from transformers import AdamW
import torch
from tqdm import tqdm

optimizer = AdamW(model_pegasus.parameters(), lr=5e-5)

num_epochs = 20
for epoch in range(num_epochs):
    model_pegasus.train()

    # Use tqdm to create a progress bar for training batches
    tqdm_train_dataloader = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)

    for batch in tqdm_train_dataloader:
        inputs = {key: batch[key].squeeze(dim=1) for key in ['input_ids', 'attention_mask']}
        labels = batch['labels'].squeeze(dim=1)

        outputs = model_pegasus(**inputs, labels=labels)
        loss = outputs.loss

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_pegasus.parameters(), 1.0)
        optimizer.step()
        optimizer.zero_grad()

        # Update the tqdm progress bar description with the current loss
        tqdm_train_dataloader.set_postfix(loss=loss.item())

    # Close the tqdm progress bar for the training epoch
    tqdm_train_dataloader.close()

print("Training complete.")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
                                                                       

Training complete.


In [15]:
output_dir = f"/content/pegasus_fine_tuned_model"
model_pegasus.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/pegasus_fine_tuned_model/tokenizer_config.json',
 '/content/pegasus_fine_tuned_model/special_tokens_map.json',
 '/content/pegasus_fine_tuned_model/spiece.model',
 '/content/pegasus_fine_tuned_model/added_tokens.json',
 '/content/pegasus_fine_tuned_model/tokenizer.json')

In [16]:
import shutil

model_directory_path = '/content/pegasus_fine_tuned_model'
zip_file_path_colab = '/content/drive/MyDrive/NLP/pegasus_fine_tuned_model.zip'
shutil.make_archive(zip_file_path_colab[:-4], 'zip', model_directory_path)
print(f"Zip file 'pegasus_fine_tuned_model.zip' created in the Colab environment.")


Zip file 'pegasus_fine_tuned_model.zip' created in the Colab environment.


In [17]:
import torch
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import pandas as pd

model_path = "/content/pegasus_fine_tuned_model"
model = PegasusForConditionalGeneration.from_pretrained(model_path)
tokenizer = PegasusTokenizer.from_pretrained(model_path)

In [ ]:
# test_summaries = []
# model.eval()
# with torch.no_grad():
#     for _, row in test_data.iterrows():
#         text = row['Text']
#         encoding = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding=True)
#         generated_ids = model.generate(encoding['input_ids'], max_length=150, num_beams=4, length_penalty=2.0, early_stopping=True)
#         generated_summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
#         test_summaries.append(generated_summary)

# # Display results for test dataset
# print("Results for Test Dataset:")
# for i, (_, row) in enumerate(test_data.iterrows()):
#     print(f"Input Text {i + 1}: {row['Text']}")
#     print(f"Generated Summary {i + 1}: {test_summaries[i]}")
#     print("="*50)

In [22]:
x_test.head()

,Summary,Text
383,Next chief executive Simon Wolfson admitted th...,Weak end-of-year sales hit Next\n\nNext has sa...
35,Many foreign firms had protested that the SEC ...,US gives foreign firms extra time\n\nForeign f...
189,"""In the beginning of 2004, growth was mainly d...",Consumers drive French economy\n\nFrance's eco...
210,Hitachi recently announced a deal to buy plasm...,Electronics firms eye plasma deal\n\nConsumer ...
231,The FSA took over regulation of the general in...,Lloyd's of London head chides FSA\n\nThe head ...


In [23]:
test_summaries = []
single_input = input()
single_input_encoding = tokenizer(single_input, return_tensors='pt', max_length=512, truncation=True, padding=True)
with torch.no_grad():
    generated_ids_single = model.generate(single_input_encoding['input_ids'], max_length=500, num_beams=4, length_penalty=2.0, early_stopping=True)
    generated_summary_single = tokenizer.decode(generated_ids_single[0], skip_special_tokens=True)
print("\nResult for Single Input:")
print(f"Input Text: {single_input}")
print(f"Generated Summary: {generated_summary_single}")


Weak end-of-year sales hit Next Next has said its annual profit will be Â£5m lower than previously expected because its end-of-year clearance sale has proved disappointing. "Clearance rates in our end-of-season sale have been below our expectations," the company said. The High Street retailer said it now expected to report annual profits of between Â£415m and Â£425m ($779m-798m). Next's shares fell more than 3% following the release of the trading statement. Next chief executive Simon Wolfson admitted that festive sales were "below where we would expect a normal Christmas to be", but said sales should still top analyst expectations. Among areas where Next could have done better, Mr Wolfson said menswear ranges were "a little bit too similar to the previous year". Mr Wolfson also said that disappointing pre-Christmas sales were "more to do with the fact that we went in with too much stock rather than (the fact that) demand wasn't there for the stock". Next's like-for-like store sales in